In [1]:
!pip install python-chi python-blazarclient python-dotenv


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [11]:
# if running on local system (not chameleon jupyter)
from dotenv import load_dotenv
import chi
load_dotenv(".env")


True

In [12]:
from chi import context
context.check_credentials()

Username: mr7375@nyu.edu
Currently site: CHI@UC
Currently project: CH-23118
Projects:
	 CHI-231138
Authentication is valid.


True

In [18]:
chi.set('project_name', 'CH-23118')
# Select your site
chi.use_site('CHI@TACC')

Now using CHI@TACC:
URL: https://chi.tacc.chameleoncloud.org
Location: Austin, Texas, USA
Support contact: help@chameleoncloud.org


In [6]:
import chi.share
import time

# Wait for credentials to be fully loaded from earlier context
# 1. Create a 200GB share (Adjust size as needed)
# Share creation might take a few moments
print("Sending request to create Manila share 'mlsys_marconi_nfs'...")
try:
    share = chi.share.create_share(size=200, name="mlsys_marconi_nfs")
    chi.share.wait_for_status(share['id'], 'available')
    print(f"Share successfully created with ID: {share['id']}")
except Exception as e:
    print("Share might already exist or creation failed. Note: you only need to run this cell once per project, not per lease.")
    print(f"Error details: {e}")
    # Try to grab the existing one if it failed because it exists
    share = chi.share.get_share("mlsys_marconi_nfs")
    
# We will grant access to this share LATER, once the node is booted and we have its IP address.


Sending request to create Manila share 'mlsys_marconi_nfs'...
Share might already exist or creation failed. Note: you only need to run this cell once per project, not per lease.
Error details: module 'chi.share' has no attribute 'wait_for_status'


In [14]:
from chi import lease
import datetime

l = lease.Lease(
    name="mlsys_marconi_mr7375",
    duration=datetime.timedelta(hours=48),
    start_date=datetime.datetime(2026, 2, 20, 13, 52, 0,0, datetime.timezone.utc)
)

# Correct for GigaIO GPU pool
l.add_node_reservation(
    amount=1,
    node_type="gpu_v100"
)

l.submit(idempotent=True, wait_for_active=False)
l.add_fip_reservation(amount=1)


error: not enough resources available with query {'resource_type': 'physical:host', 'resource_properties': '["==", "$node_type", "gpu_v100"]', 'hypervisor_properties': '', 'min': 1, 'max': 1, 'start_date': datetime.datetime(2026, 2, 20, 13, 52), 'end_date': datetime.datetime(2026, 2, 21, 16, 28), 'project_id': 'c819e73b9bfd4c0595bf49dcf338d51b', 'count_range': '1-1', 'before_end': 'default', 'on_start': 'default'}


ResourceError: Unable to make lease

In [21]:
l = lease.Lease(
    name="mlsys_mr7375_a100_week7",
    # duration=datetime.timedelta(hours=48),
    start_date=datetime.datetime(2026, 3, 8, 00, 00, 0,0, datetime.timezone.utc),
    end_date=datetime.datetime(2026, 3, 13, 23, 30, 0,0, datetime.timezone.utc)

)

# Correct for GigaIO GPU pool
l.add_node_reservation(
    amount=1,
    node_type="compute_gigaio"
)

l.submit(idempotent=True, wait_for_active=False)
l.add_fip_reservation(amount=1)

The python binding code in neutronclient is deprecated in favor of OpenstackSDK, please use that as this will be removed in a future release.


In [24]:
from chi import lease
import datetime

storage_lease = lease.Lease(
    name="mlsys_marconi_storage_net",
    # Match your existing node lease's time window
    start_date=datetime.datetime.now(datetime.timezone.utc),
    end_date=datetime.datetime(2026, 2, 26, 0, 0, 0, 0, datetime.timezone.utc)
)
storage_lease.add_network_reservation(
    network_name="mlsys_storage_net",
    usage_type="storage"
)
storage_lease.submit(idempotent=True)


Waiting for lease to start...


Lease mlsys_marconi_storage_net has reached status active


In [25]:
from chi import network, clients

# Get the reserved storage network
storage_net = network.get_network("mlsys_storage_net")
storage_net_id = storage_net["id"]

# Use Nova client to attach the interface to the running server
nova = clients.nova()
nova.servers.interface_attach(
    server=s.id,       # your existing Server object's ID
    net_id=storage_net_id,
    port_id=None,
    fixed_ip=None
)

# Refresh to see the new address
import time
time.sleep(10)
s.refresh()
print(s.addresses)


The python binding code in neutronclient is deprecated in favor of OpenstackSDK, please use that as this will be removed in a future release.
The python binding code in neutronclient is deprecated in favor of OpenstackSDK, please use that as this will be removed in a future release.


ClientException: Failed to attach network adapter device to 611506ce-ee46-470d-9a3e-4910cb1f52e8 (HTTP 500) (Request-ID: req-3040f9d7-04d9-463f-b97a-0bfa5f1cc5f9)

In [2]:
from chi import server
s = server.Server('mlsys_marconi_mr7375',
                #   reservation_id=l.node_reservations[0]["id"],
                reservation_id="df41f2e4-1261-414f-b7cc-a646e3afc99a",
                image_name="CC-Ubuntu22.04-CUDA",
                flavor_name="baremetal",
                key_name="sparkle",
                network_name="sharednet1"
                  
)
s.submit(idempotent=True)

s.associate_floating_ip()


Waiting for server mlsys_marconi_mr7375's status to become ACTIVE. This typically takes 10 minutes for baremetal, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,mlsys_marconi_mr7375
Id,611506ce-ee46-470d-9a3e-4910cb1f52e8
Status,ACTIVE
Image Name,CC-Ubuntu22.04-CUDA
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.140.81.218 (v4) Type: fixed MAC: 14:23:f2:a3:f8:00
Network Name,None
Created At,2026-02-24T05:29:56Z
Keypair,sparkle
Reservation Id,None
Host Id,7cbd4d17ddcda48102257306b0d7d2bcee03b631dcd5ebd809bc0fff


The python binding code in neutronclient is deprecated in favor of OpenstackSDK, please use that as this will be removed in a future release.


'192.5.86.151'

In [4]:
s.check_connectivity()

Checking connectivity to 192.5.86.151 port 22.


Connection successful


In [8]:
import os
connect_kwargs = {"key_filename": os.getenv("KEY_PATH")}

In [ ]:
s.execute("sudo apt update", connect_kwargs=connect_kwargs)


/Users/megh/nyu/research/mlsys-debloat/.venv/lib/python3.14/site-packages/paramiko/client.py:885: UserWarning: Unknown ssh-ed25519 host key for 129.114.108.180: b'5efbb6f9095060af3718b85640ccbd36'
  warnings.warn(




Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1581 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2361 kB]
Hit:4 http://nova.clouds.archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2939 kB]
Get:7 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3217 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 c-n-f Metadata [14.1 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [5032 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 c-n-f Metadata [680 B]
Get:12 http://security.ubunt

<Result cmd='sudo apt update' exited=0>

In [9]:
s.execute("curl -sSL https://get.docker.com/ | sudo sh", connect_kwargs=connect_kwargs)
s.execute("sudo groupadd -f docker; sudo usermod -aG docker $USER", connect_kwargs=connect_kwargs)
     

s.execute("curl -fsSL https://nvidia.github.io/libnvidia-container/gpgkey | sudo gpg --dearmor -o /usr/share/keyrings/nvidia-container-toolkit-keyring.gpg \
  && curl -s -L https://nvidia.github.io/libnvidia-container/stable/deb/nvidia-container-toolkit.list | \
    sed 's#deb https://#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://#g' | \
    sudo tee /etc/apt/sources.list.d/nvidia-container-toolkit.list", connect_kwargs=connect_kwargs)
s.execute("sudo apt update", connect_kwargs=connect_kwargs)
s.execute("sudo apt-get install -y nvidia-container-toolkit", connect_kwargs=connect_kwargs)
s.execute("sudo nvidia-ctk runtime configure --runtime=docker", connect_kwargs=connect_kwargs)
# for https://github.com/NVIDIA/nvidia-container-toolkit/issues/48
s.execute("sudo jq 'if has(\"exec-opts\", connect_kwargs=connect_kwargs) then . else . + {\"exec-opts\": [\"native.cgroupdriver=cgroupfs\"]} end' /etc/docker/daemon.json | sudo tee /etc/docker/daemon.json.tmp > /dev/null && sudo mv /etc/docker/daemon.json.tmp /etc/docker/daemon.json", connect_kwargs=connect_kwargs)
s.execute("sudo systemctl restart docker", connect_kwargs=connect_kwargs)


s.execute("sudo apt update", connect_kwargs=connect_kwargs)
s.execute("sudo apt -y install nvtop", connect_kwargs=connect_kwargs)

/Users/megh/nyu/research/mlsys-marconi/.venv/lib/python3.14/site-packages/paramiko/client.py:885: UserWarning: Unknown ssh-ed25519 host key for 192.5.86.151: b'a0056ea479a1fa8536a8df909219d696'
  warnings.warn(


# Executing docker install script, commit: f381ee68b32e515bb4dc034b339266aff1fbc460


+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install ca-certificates curl >/dev/null
Use of uninitialized value $ucode_vars{"AVAIL"} in concatenation (.) or string at /usr/sbin/needrestart line 942.
+ sh -c install -m 0755 -d /etc/apt/keyrings
+ sh -c curl -fsSL "https://download.docker.com/linux/ubuntu/gpg" -o /etc/apt/keyrings/docker.asc
+ sh -c chmod a+r /etc/apt/keyrings/docker.asc
+ sh -c echo "deb [arch=amd64 signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu jammy stable" > /etc/apt/sources.list.d/docker.list
+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install docker-ce docker-ce-cli containerd.io docker-compose-plugin docker-ce-rootless-extras docker-buildx-plugin docker-model-plugin >/dev/null
Use of uninitialized value $ucode_vars{"AVAIL"} in concatenation (.) or string at /usr/sbin/needrestart line 942.
Using systemd to manage Docker service
+ sh -c sy

  UNIT                                                                                                  LOAD   ACTIVE SUB       DESCRIPTION
  proc-sys-fs-binfmt_misc.automount                                                                     loaded active running   Arbitrary Executable File Formats File System Automount Point
  sys-bus-pci-drivers-nvidia.device                                                                     loaded active plugged   /sys/bus/pci/drivers/nvidia
  sys-devices-pci0000:78-0000:78:03.1-0000:7b:00.0-net-eno12399np0.device                               loaded active plugged   BCM57504 NetXtreme-E 10Gb/25Gb/40Gb/50Gb/100Gb/200Gb Ethernet (NetXtreme-E BCM57504 4x25G OCP3.0)
  sys-devices-pci0000:78-0000:78:03.1-0000:7b:00.1-net-eno12409np1.device                               loaded active plugged   BCM57504 NetXtreme-E 10Gb/25Gb/40Gb/50Gb/100Gb/200Gb Ethernet (NetXtreme-E BCM57504 4x25G OCP3.0)
  sys-devices-pci0000:78-0000:78:03.1-0000:7b:00.2-net-eno1241

 --now docker.service


...

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://nvidia.github.io/libnvidia-container/stable/deb/amd64  InRelease
Hit:3 https://download.docker.com/linux/ubuntu jammy InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://nova.clouds.archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists...
Building dependency tree...
Reading state information...
141 packages can be upgraded. Run 'apt list --upgradable' to see them.


Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  nvtop
0 upgraded, 1 newly installed, 0 to remove and 141 not upgraded.
Need to get 43.9 kB of archives.
After this operation, 106 kB of additional disk space will be used.
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu jammy/multiverse amd64 nvtop amd64 1.2.2-1 [43.9 kB]


debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 43.9 kB in 1s (79.8 kB/s)
Selecting previously unselected package nvtop.
(Reading database ... 92514 files and directories currently installed.)
Preparing to unpack .../nvtop_1.2.2-1_amd64.deb ...
Unpacking nvtop (1.2.2-1) ...
Setting up nvtop (1.2.2-1) ...
Processing triggers for man-db (2.10.2-1) ...

Running kernel seems to be up-to-date.

The processor microcode seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.


<Result cmd='sudo apt -y install nvtop' exited=0>

In [11]:
s.execute("git clone --recurse-submodules https://github.com/meghrathod/marconi-repro", connect_kwargs=connect_kwargs)

/Users/megh/nyu/research/mlsys-marconi/.venv/lib/python3.14/site-packages/paramiko/client.py:885: UserWarning: Unknown ssh-ed25519 host key for 192.5.86.151: b'a0056ea479a1fa8536a8df909219d696'
  warnings.warn(
Cloning into 'marconi-repro'...
Submodule 'marconi' (https://github.com/ruipeterpan/marconi) registered for path 'marconi'
Cloning into '/home/cc/marconi-repro/marconi'...
From https://github.com/ruipeterpan/marconi
 * branch            90161fbe17020ba53b8e42f006a3b6657cac162d -> FETCH_HEAD


Submodule path 'marconi': checked out '90161fbe17020ba53b8e42f006a3b6657cac162d'


<Result cmd='git clone --recurse-submodules https://github.com/meghrathod/marconi-repro' exited=0>

In [18]:
print(s.addresses)

{'sharednet1': [{'version': 4, 'addr': '10.140.81.218', 'OS-EXT-IPS:type': 'fixed', 'OS-EXT-IPS-MAC:mac_addr': '14:23:f2:a3:f8:00'}, {'version': 4, 'addr': '192.5.86.151', 'OS-EXT-IPS:type': 'floating', 'OS-EXT-IPS-MAC:mac_addr': '14:23:f2:a3:f8:00'}]}


In [21]:
import chi.storage

chi.storage.get_share(ref="mlsys_marconi_nfs")



TypeError: Share._from_manilla_share() missing 1 required positional argument: 'share'

In [ ]:
from chi import clients
import time

manila = clients.manila()
share_name = "mlsys_marconi_nfs"

# Ensure we have our share reference
shares = manila.shares.list(search_opts={"name": share_name})
if not shares:
    raise Exception(f"Could not find share '{share_name}'")
share = shares[0]

# 1. Get your instance's internal IP address on the 'sharednet1' network
try:
    private_ip = s.addresses['sharednet1'][0]['addr']
except KeyError:
    print("Error: Could not find 'sharednet1' IP address. Is the network named differently?")
    raise

# 2. Allow your instance to access the share using OpenStack Manila API
print(f"Allowing NFS access for node IP: {private_ip}")
try:
    manila.shares.allow(share, access_type="ip", access=private_ip, access_level="rw")
    time.sleep(5)  # Give OpenStack time to apply the ACL rules
except Exception as e:
    # Manila throws a 400 Bad Request if the access rule already exists. We can safely ignore it.
    print(f"Access rule likely already exists. Proceeding... ({e})")

# 3. Grab the exact network path to mount
share = manila.shares.get(share.id) # Refresh to get export locations
export_location = share.export_locations[0]
print(f"Target Share Export Location: {export_location}")

# 4. Mount it on the remote instance
print("Mounting NFS on bare-metal node...")
s.execute("sudo apt-get install -y nfs-common", connect_kwargs=connect_kwargs)
s.execute("sudo mkdir -p /mnt/marconi_nfs", connect_kwargs=connect_kwargs)
s.execute(f"sudo mount -t nfs {export_location} /mnt/marconi_nfs", connect_kwargs=connect_kwargs)




Allowing NFS access for node IP: 10.140.81.218
Access rule likely already exists. Proceeding... (Access was denied to this resource. (HTTP 403) (Request-ID: req-1b91b06e-b828-4827-aa6f-9038abf04ebb))
Target Share Export Location: 10.31.0.14:/volumes/_nogroup/85daa9e3-53f5-4ee2-9db6-72491789efab/471e791c-e61f-4f7d-a268-71c42ca757b6
Mounting NFS on bare-metal node...


/Users/megh/nyu/research/mlsys-marconi/.venv/lib/python3.14/site-packages/paramiko/client.py:885: UserWarning: Unknown ssh-ed25519 host key for 192.5.86.151: b'a0056ea479a1fa8536a8df909219d696'
  warnings.warn(


Reading package lists...
Building dependency tree...
Reading state information...
nfs-common is already the newest version (1:2.6.1-1ubuntu1.2).
0 upgraded, 0 newly installed, 0 to remove and 141 not upgraded.


mount.nfs: Connection timed out


KeyboardInterrupt: 

In [26]:
# Add to fstab for persistence across reboots during the lease
fstab_entry = f"{export_location} /mnt/marconi_nfs nfs defaults 0 0"
s.execute(f"echo '{fstab_entry}' | sudo tee -a /etc/fstab", connect_kwargs=connect_kwargs)

# 5. Reconfigure Docker
print("Migrating Docker data-root to NFS...")
s.execute("sudo systemctl stop docker", connect_kwargs=connect_kwargs)

docker_daemon_json = '{"data-root": "/mnt/marconi_nfs/docker"}'
s.execute(f"echo '{docker_daemon_json}' | sudo tee /etc/docker/daemon.json", connect_kwargs=connect_kwargs)

s.execute("sudo systemctl start docker", connect_kwargs=connect_kwargs)

# 6. Configure Hugging Face Cache
print("Configuring Hugging Face cache directory...")
s.execute("sudo mkdir -p /mnt/marconi_nfs/hf_cache", connect_kwargs=connect_kwargs)
s.execute("sudo chmod 777 /mnt/marconi_nfs/hf_cache", connect_kwargs=connect_kwargs)
s.execute("echo 'export HF_HOME=/mnt/marconi_nfs/hf_cache' >> ~/.bashrc", connect_kwargs=connect_kwargs)

print("✅ NFS Storage successfully attached and Docker/HF configured!")

/Users/megh/nyu/research/mlsys-marconi/.venv/lib/python3.14/site-packages/paramiko/client.py:885: UserWarning: Unknown ssh-ed25519 host key for 192.5.86.151: b'a0056ea479a1fa8536a8df909219d696'
  warnings.warn(


10.31.0.14:/volumes/_nogroup/85daa9e3-53f5-4ee2-9db6-72491789efab/471e791c-e61f-4f7d-a268-71c42ca757b6 /mnt/marconi_nfs nfs defaults 0 0
Migrating Docker data-root to NFS...


  docker.socket


{"data-root": "/mnt/marconi_nfs/docker"}
Configuring Hugging Face cache directory...
✅ NFS Storage successfully attached and Docker/HF configured!
